## Homework
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.
### Q1. Downloading the data
[The data link address of NYC Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

Read the data for January. How many columns are there?

In [1]:
import pandas as pd
nyc_dataframe = pd.read_parquet("../data/yellow_tripdata_2022-01.parquet")

In [5]:
print("Number of columns are: ", nyc_dataframe.shape[1])

Number of columns are:  19
